In [ ]:
import scvelo as scv
import scanpy as sc
import os
import pandas as pd
from scipy.stats import rankdata 
import numpy as np

# scv.set_figure_params()
scv.set_figure_params(
    style='scvelo', 
    dpi=100, 
    dpi_save=300, 
    vector_friendly=False 
)


folder = os.path.join('/data/work/output/PlantPhone/Seurat/6phase/RNA_T_0.5/36clusters/Annotation_21tissues') 
file1 = os.path.join(folder, 'Annotated_' + sample + '_RNA_T_0.5_unsplicedlayer.h5ad')
file2 = os.path.join(folder, 'Annotated_' + sample + '_RNA_T_0.5_splicedassay.loom')

output = '/data/work/output/Trajectory/scVelo' 

folder_path = os.path.join(output, method, sample)

os.makedirs(folder_path, exist_ok=True)

adata = sc.read(file1) 
adata

sdata = sc.read_loom(file2)
sdata

sdata.layers["spliced"] = sdata.X
sdata

adata = scv.utils.merge(adata, sdata)
adata

path0 = os.path.join(folder_path, sample +"_3layers.h5ad")
adata.write(path0, compression='gzip')

## Preprocess the Data

import warnings
warnings.filterwarnings("ignore")

scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)

scv.pp.filter_and_normalize(adata)
scv.pp.moments(adata)

## Estimate RNA velocity

scv.tl.velocity(adata, mode='stochastic')

scv.tl.velocity_graph(adata)

adata

## Project the velocities

mycolor1 <- c(
  '#E41A1C', '#FF7F00', '#4DAF4A', '#984EA3', '#FFFF33',  
  '#377EB8', '#A65628', '#F781BF', '#66C2A5', '#FC8D62',  
  '#8DA0CB', '#E78AC3', '#A6D854', '#FFD92F', '#E5C494',    
  '#B3B3B3', '#1B9E77', '#D95F02', '#7570B3', '#E7298A',    
  '#66A61E', '#E6AB02', '#A6761D', '#666666', '#A6CEE3',    
  '#1F78B4', '#B2DF8A', '#33A022', '#8C564B', '#17BECF',    
  '#BCBD22', '#9467BD',                                    
  '#FF8C00',  
  '#FF4500',   
  '#FF1493',    
  '#DA70D6',  
  '#8B0000',   
  '#4682B4',    
  '#800080',    
  '#00CED1',  
  '#FFD700',    
  '#8B4513',    
  '#C71585',    
  '#FF6347',    
  '#00FF7F'     
)

mycolor = ['#1f77b4', '#ff7f0e', '#c7c7c7', '#d62728', '#aa40fc', 
                     '#8c564b', '#e377c2', '#b5bd61', '#279e68', '#aec7e8',
                     '#ffbb78',  '#bd9e39','#5254a3',  '#6b6ecf', '#ad494a',
                         '#9edae5','#dbdb8d', '#E7298A', '#f7b6d2','#8c6d31','#BCBD22']


scv.pl.velocity_embedding(adata, color='assign.ident')

scv.pl.velocity_embedding_grid(adata, basis='umap',color='assign.ident')


path1 = os.path.join(folder_path, sample +"_annotation_stochastical_stream.svg")
scv.pl.velocity_embedding_stream(
    adata, 
    basis='umap',
    color='assign.ident',
    title='Annotation', 
    save=path1, 
    legend_fontsize=10,
    palette=mycolor,
    fontsize=14,
    dpi=300
)


path2 = os.path.join(folder_path, sample +"_cluster_stochastical_stream.svg")
scv.pl.velocity_embedding_stream(
    adata, 
    basis='umap',
    palette=mycolor1,
    color='seurat_clusters',
    title='Clusters', 
    save=path2, 
    legend_fontsize=10,
    fontsize=14,
    dpi=300
)


path3 = os.path.join(folder_path, sample +"_proportions.pdf")
scv.pl.proportions(
    adata,
    save=path3, 
    dpi=300
)

## Identify important genes

scv.tl.rank_velocity_genes(adata, groupby='seurat_clusters', min_corr=.3)

df = pd.DataFrame(adata.uns['rank_velocity_genes']['names'])
df.head()

G2M = pd.read_csv('/data/work/input/DoubletFinder/cell_cycles_ITAG4.1_G2M_241212.csv', header=0, skiprows=[0], usecols=[0])

S = pd.read_csv('/data/work/input/DoubletFinder/cell_cycles_ITAG4.1_S_241212.csv', header=0, skiprows=[0], usecols=[0])

scv.tl.score_genes_cell_cycle(adata, s_genes=list(S), g2m_genes=list(G2M))

scv.pl.scatter(
    adata, 
    color_gradients=['S.Score', 'G2M.Score'], 
    smooth=True, 
    perc=[5, 95],
    title='cell cycle phase'
    # save=path4, 
    # dpi=300
)

scv.pl.scatter(
    adata, 
    color_gradients=['S_score', 'G2M_score'], 
    smooth=True, 
    perc=[5, 95],
    title='cell cycle phase'
    # save=path4, 
    # dpi=300
)

path4_1 = os.path.join(folder_path, sample +"_cell_cycle_phase_seurat.png")

col = (adata.obs['S.Score'] - adata.obs['G2M.Score']) * (adata.obs['S.Score'] + adata.obs['G2M.Score'] > 1e-2)
col /= np.max(np.abs(col))

ax = scv.pl.scatter(adata, color='lightgrey', show=False, alpha=1, size=200)

scv.pl.scatter(adata, color=col, color_map='RdBu', vmid=0, alpha=1, smooth=True, size=80, title='cell cycle phase', save=path4_1, ax=ax, zorder=3)

path4_2 = os.path.join(folder_path, sample +"_cell_cycle_phase_scv.png")

col = (adata.obs['S_score'] - adata.obs['G2M_score']) * (adata.obs['S_score'] + adata.obs['G2M_score'] > 1e-2)
col /= np.max(np.abs(col))

ax = scv.pl.scatter(adata, color='lightgrey', show=False, alpha=1, size=200)

scv.pl.scatter(adata, color=col, color_map='RdBu', vmid=0, alpha=1, smooth=True, size=80, title='cell cycle phase', save=path4_2, ax=ax, zorder=3)

## Speed and coherence

path5 = os.path.join(folder_path, sample +"_velocity_length_confidence.png")
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(
    adata, 
    c=keys, 
    cmap='coolwarm', 
    perc=[5, 95],
    save=path5, 
    dpi=300
)

df = adata.obs.groupby('seurat_clusters')[list(keys)].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

## Velocity graph and pseudotime

path6 = os.path.join(folder_path, sample +"_velocity_graph.pdf")
scv.pl.velocity_graph(
    adata,
    color='assign.ident',
    threshold=.1,
    title='Annotation',
    save=path6, 
    dpi=300
)


path6_1 = os.path.join(folder_path, sample +"_anscestors.png")

x, y = scv.utils.get_cell_transitions(adata, basis='umap', starting_cell=0) 

ax = scv.pl.velocity_graph(
    adata, 

    c='lightgrey', 
    edge_width=.05, 
    show=False)

ax = scv.pl.scatter(
    adata, 
    color='assign.ident',
    x=x, 
    y=y, 
    s=50, 
    c='ascending', 
    cmap='gnuplot', 
    ax=ax,
    save=path6_1, 
    dpi=300
)


scv.tl.velocity_pseudotime(adata)


path7 = os.path.join(folder_path, sample +"_velocity_pseudotime.png")
scv.pl.scatter(
    adata, 
    color='velocity_pseudotime', 
    cmap='gnuplot',
    save=path7, 
    dpi=300
)

## PAGA velocity graph

scv.tl.paga(adata, groups='assign.ident')

df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')

path8_1 = os.path.join(folder_path, sample +"_paga_annotation.pdf")
scv.pl.paga(
    adata, 
    basis='umap', 
    color='assign.ident',
    palette=mycolor,
    size=50, 
    alpha=.1,
    min_edge_width=2, 
    node_size_scale=1.5,
    save=path8_1, 
    dpi=300
)


adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

scv.tl.paga(adata, groups='seurat_clusters')
df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')


path8_2 = os.path.join(folder_path, sample +"_paga_clusters.pdf")
scv.pl.paga(
    adata, 
    basis='umap', 
    palette=mycolor1,
    color='seurat_clusters',
    legend_loc='on data',
    size=50, 
    alpha=.1,
    min_edge_width=2, 
    node_size_scale=1.5,
    save=path8_2, 
    dpi=300
)

## Save 

# save data

path9 = os.path.join(folder_path, sample +"_scvelo_stochastical.h5ad")
adata.write(path9, compression='gzip')
adata

## Dynamical Model

We run the dynamical model to learn the full transcriptional dynamics of splicing kinetics.
It thereby aims to learn the unspliced/spliced phase trajectory for each gene

scv.tl.recover_dynamics(adata)

scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

# save

path17 = os.path.join(folder_path, sample +"_scvelo_dynamical.h5ad")
adata.write(path17, compression='gzip')
adata

# Dynamical stream

path10 = os.path.join(folder_path, sample +"_annotation_Dynamical_stream.svg")
scv.pl.velocity_embedding_stream(
    adata, 
    basis='umap', 
    color='assign.ident',
    title='Annotation', 
    save=path10, 
    legend_fontsize=10,
    fontsize=14,
    dpi=300
)

## Kinetic rate paramters

# The rates of RNA transcription, splicing and degradation are estimated without the need of any experimental data.

df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)

scv.get_df(adata, 'fit*', dropna=True).head()

scv.tl.latent_time(adata)



path11 = os.path.join(folder_path, sample +"_Dynamical_latent_time.png")
scv.pl.scatter(
    adata, 
    color='latent_time', 
    fontsize=24, 
    size=100,
    color_map='gnuplot', 
    perc=[2, 98], 
    colorbar=True, 
    rescale_color=[0,1],
    save=path11, 
    dpi=300
)


top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]

# Dynamical topgenes

path12 = os.path.join(folder_path, sample +"_Dynamical_topgenes.png")

scv.pl.heatmap(
    adata, 
    var_names=top_genes, 
    sortby='latent_time', 
    col_color='assign.ident', 
    n_convolve=100,
    save=path12
    # dpi=300
)

## Top-likelihood genes

# Dynamical_top_likehood_genes

path13 = os.path.join(folder_path, sample +"_Dynamical_top_likehood_genes.pdf")
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(
    adata, 
    basis=top_genes[:15], 
    ncols=5, 
    color='assign.ident',
    frameon=False,
    save=path13,
    dpi=300
)

## Cluster-specific top-likelihood genes

scv.tl.rank_dynamical_genes(adata, groupby='assign.ident')
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(5)

path13_3 = os.path.join(folder_path, sample + "_Dynamical_cluster_likehood_genes.csv")

df.to_csv(path13_3, index=False)

folder_path1 = os.path.join(output, method, sample, 'top_likehood_genes')
os.makedirs(folder_path1, exist_ok=True)

for cluster in adata.obs['assign.ident'].unique():
    safe_cluster = cluster.replace('/', '_')
    path13_2 = os.path.join(folder_path1, sample + '_' + safe_cluster + "_Dynamical_top_likehood_genes.pdf")
    scv.pl.scatter(adata, df[cluster][:5], ylabel=cluster, frameon=False, color='assign.ident', save=path13_2, dpi=300)

# pseudotime
path13_1 = os.path.join(folder_path, sample +"_Dynamical_pseudotime.png")

adata.uns['iroot'] = adata.obsm['X_umap'][:, 0].argmin()
sc.tl.diffmap(adata)
sc.tl.dpt(adata)

scv.pl.scatter(adata, color='dpt_pseudotime', title='pseudotime', fontsize=24, size=100,
               color_map='gnuplot', perc=[2, 98], colorbar=True, rescale_color=[0,1],save=path13_1,dpi=300)

## Differential Kinetics


scv.tl.velocity(adata, diff_kinetics=True)
scv.tl.velocity_graph(adata)

scv.pl.velocity_embedding(adata, dpi=120, arrow_size=2, arrow_length=2,  color='assign.ident')

# diff_kinetics_annotation

path14 = os.path.join(folder_path, sample +"_diff_kinetics_annotation_stream.svg")
scv.pl.velocity_embedding_stream(
    adata, 
    basis='umap',
    color='assign.ident',
    title='Annotation', 
    save=path14, 
    legend_fontsize=10,
    fontsize=14,
    dpi=300
)


# black background

path15 = os.path.join(folder_path, sample +"_blackcover_stream.svg")

kwargs = dict(color='latent_time', color_map='gnuplot', title='',
              vmin=-.1, colorbar=False, show=False, dpi=300)

ax = scv.pl.velocity_embedding(adata, arrow_length=4.3, arrow_size=1.8, alpha=.5, **kwargs)

ax = scv.pl.velocity_embedding_stream(adata, alpha=.01, arrow_color='lightgrey', ax=ax, save = path15,
                                      smooth=.7, min_mass=3.5, density=.6, linewidth=.5, **kwargs)

# white background
path16 = os.path.join(folder_path, sample +"_whitecover_stream.svg")

kwargs = dict(color='latent_time', color_map='gnuplot', title='', vmin=-.1, colorbar=False, show=False, dpi=300)

ax = scv.pl.velocity_embedding(adata, arrow_length=4, arrow_size=1.5, alpha=.5, **kwargs)

ax = scv.pl.velocity_embedding_stream(adata, size=150, alpha=.01, arrow_color='white', ax=ax,
                                      smooth=.7, min_mass=3.5, density=.6, linewidth=.6, **kwargs)

ax = scv.pl.velocity_embedding_stream(adata, size=150, alpha=.1, arrow_color='darkslategrey', ax=ax, save=path16,
                                      smooth=.7, min_mass=3.5, density=.6, linewidth=.5, **kwargs)

# save

path17 = os.path.join(folder_path, sample +"_scvelo_dynamical.h5ad")
adata.write(path17, compression='gzip')
adata